# Data Cleaning

## Cleaning Steps
Document every cleaning step here.

In [2]:
import pandas as pd, pathlib
from pandas import json_normalize

raw = pathlib.Path('../data/raw/stops.json')
df_raw = pd.read_json(raw)

# Rozbal stopGroups
df = json_normalize(df_raw['stopGroups'])

# 👉 zjisti, které z těchto sloupců existují
id_col_candidates = ['idosCode', 'idosId', 'idosIds', 'uuid']
id_col = next((c for c in id_col_candidates if c in df.columns), None)
print("Budu deduplikovat podle:", id_col)

# 1) Odstraníme duplicity
if id_col:
    df = df.drop_duplicates(subset=id_col)

# 2) Sloučíme/oddělíme souřadnice – bývají v nested kolonkách
if 'gps.latitude' in df.columns:
    df['latitude']  = df['gps.latitude'].astype(float)
    df['longitude'] = df['gps.longitude'].astype(float)
elif {'gpsLatitude', 'gpsLongitude'}.issubset(df.columns):
    df = df.rename(columns={'gpsLatitude': 'latitude',
                            'gpsLongitude': 'longitude'})

# 3) Přehledné přejmenování pro portfolio
rename_map = {'name': 'stop_name',
              'districtCode': 'district_code'}
df = df.rename(columns={k: v for k, v in rename_map.items() if k in df.columns})

# 4) Ulož hotová data
clean_path = pathlib.Path('../data/clean/stops_clean.xlsx')
df.to_excel(clean_path, index=False)
print(f'✅  Saved cleaned file → {clean_path.resolve()}')

Budu deduplikovat podle: None
✅  Saved cleaned file → /workspaces/jupiter-dev/data/clean/stops_clean.xlsx
